In [1]:
import pandas as pd
import pickle
import numpy as np
import torch
from torch import nn
import os 
import fastai
import fastai
from fastai.text import *
PATH="/home/kirana/Documents/phd/final"
DATAPATH="/home/kirana/Documents/phd/data/aclImdb"

## Dataset

In [2]:
[df_train,df_valid,df_test,itos, train_tokens, valid_tokens, test_tokens, trn_lm, val_lm, test_lm]=pickle.load(open(f'{PATH}/inter/dfs_tokens_fastai.pkl','rb'))

In [3]:
type(df_train)

pandas.core.frame.DataFrame

In [4]:
df_train=pd.concat([df_train,df_valid],axis=0)

In [5]:
df_train.shape

(75000, 4)

In [6]:
df_valid=df_test.copy()

In [7]:
df_train=df_train.loc[df_train['label']<2]

In [8]:
df_train.shape

(25000, 4)

In [9]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [10]:
type(df_train['tokens'])

pandas.core.series.Series

In [11]:
bs=52 # 52 - Jeremey, 20 - default

In [12]:
bptt= 70 #70 - Jeremey, 35 - default

In [13]:
df_train.head()

,label,text,words,tokens
3218,0,I've noticed how all the other reviews of this...,"[ \n , xxbos, xxfld, 1, i, 've, noticed, how, ...","[41, 42, 43, 40, 13, 161, 2030, 110, 44, 3, 10..."
6854,0,I went on a visit to one of my relatives a whi...,"[ \n , xxbos, xxfld, 1, i, went, on, a, visit,...","[41, 42, 43, 40, 13, 436, 30, 7, 2074, 9, 39, ..."
2697,0,Darcy and her young daughter Pamela are headin...,"[ \n , xxbos, xxfld, 1, xxmaj, darcy, and, her...","[41, 42, 43, 40, 2, 9225, 6, 55, 207, 551, 2, ..."
5747,0,Battlestar Gallactica was so great because it ...,"[ \n , xxbos, xxfld, 1, xxmaj, battlestar, xxm...","[41, 42, 43, 40, 2, 7900, 2, 33242, 20, 52, 10..."
8731,0,"Any movie with ""National Lampoon"" in the title...","[ \n , xxbos, xxfld, 1, xxmaj, any, movie, wit...","[41, 42, 43, 40, 2, 120, 25, 23, 16, 2, 2155, ..."


In [14]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [15]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [16]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [17]:
padlen=1400
padding_idx=1

In [18]:
df_train['n_tok']=df_train['tokens'].apply(len)

In [19]:
df_train['n_tok'].describe()

count    25000.000000
mean       304.945840
std        224.981807
min         16.000000
25%        166.000000
50%        228.000000
75%        371.000000
max       3354.000000
Name: n_tok, dtype: float64

In [20]:
df_train.sort_values(by='n_tok', inplace=True)

In [21]:
df_valid['n_tok']=df_valid['tokens'].apply(len)
df_valid.sort_values(by='n_tok', inplace=True)

In [22]:
df_test['n_tok']=df_test['tokens'].apply(len)
df_test.sort_values(by='n_tok', inplace=True)

In [23]:
for i in range(0,20):
    x=df_train['tokens'].values[i]
    y=df_train['label'].values[i]    
    out=np.ones(padlen)
    if len(x) < padlen:
        out[:len(x)]=x
    else:
        out=x[:padlen]
    print (out)
    print (y)
    print (len(out))
    if i>10:
        break

[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
1
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400
[41. 42. 43. 40. ...  1.  1.  1.  1.]
0
1400


In [24]:
df_train.shape

(25000, 5)

In [25]:
df_train['label'].value_counts(), df_valid['label'].value_counts(),df_test['label'].value_counts()

(1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64, 1    12500
 0    12500
 Name: label, dtype: int64)

In [26]:
df_train.shape, df_valid.shape

((25000, 5), (25000, 5))

In [27]:
padlen

1400

In [28]:
class ds_sentiment (Dataset):
    def __init__ (self,df,bs,padlen=64,xvar='tokens',yvar='label',len_var='n_tok',padding_idx=1):
        self.x,self.y,self.padlen,self.padding_idx,self.len_var,self.bs=\
            df[xvar],df[yvar],padlen,padding_idx,df[len_var],bs
        self.len_var=self.len_var.clip(0,padlen)
    
    def pad (self,x):
        out=np.ones(self.padlen)*self.padding_idx
        out=out.astype(int)
        if len(x)>=self.padlen:
            out[:]=x[:self.padlen]
        else:
            out[:len(x)]=x
        return out
        
    def __getitem__(self,idx):
        #import pdb
        #pdb.set_trace()
        return self.pad(self.x.iloc[idx]),self.y.iloc[idx],self.len_var.iloc[idx]
    
    def __len__(self):
        return len(self.x)        


In [29]:
#padlen=max(df_train['n_tok'])
padlen=1400

In [30]:
padlen

1400

In [31]:
dstrain=ds_sentiment(df_train,bs,padlen)
dsvalid=ds_sentiment(df_valid,bs,padlen)
dstest=ds_sentiment(df_test,bs,padlen)

In [32]:
dltrain=DataLoader(dstrain,bs,True)
dlvalid=DataLoader(dsvalid,bs,False)
dltest=DataLoader(dstest,bs,False)

In [33]:
for xb,yb,xlen in dltrain:
    break

In [34]:
xb, yb, xlen

(tensor([[41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         ...,
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1],
         [41, 42, 43,  ...,  1,  1,  1]]),
 tensor([0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
         0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
         1, 1, 1, 0]),
 tensor([ 336,  201,  728,   61,  287,  370,  243,  234,  760,  307,  160,  229,
          859,  836,  265,  255,  367,  828,  155,  407,  186,   90, 1006,  159,
          153,  150,  495,  465,  658,  231,  140,  177,  155,   44,  150,  191,
          512,  264,  197,   51,  301,  279, 1010,  170,  344,  696,  258,  166,
          161,  399,  408,  175]))

## Model Architecture

In [35]:
from torch import nn
import torch
from torch.autograd import Variable

In [36]:
bs,bptt

(52, 70)

In [37]:
n_inp=len(itos)
n_emb=400 #650
n_hidden=400#400
n_layers= 2 # 2
dropout=0.5 # 0.5
wd=1e-6
bidirectional=True
dropout_e=0.2 # 0.5
dropout_o=0.5 #0.5
n_out=1

In [38]:
xb.shape,xlen.shape

(torch.Size([52, 1400]), torch.Size([52]))

In [39]:
df_train['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [40]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [41]:
itos[:5]

['_unk_', '_pad_', 'xxmaj', 'the', '.']

In [42]:
import torch

In [43]:
m = nn.AdaptiveAvgPool1d(5)
input = torch.randn(1, 64, 8)
input.size()

torch.Size([1, 64, 8])

In [44]:
m(input).size()

torch.Size([1, 64, 5])

In [112]:
class sentiment_classifier (nn.Module):
    def __init__(self,n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=0.05,dropout=0.5,\
                 dropout_o=0.5,pretrain_mtx=None,n_out=1,padding_idx=1,n_filters=100,filter_sizes=[3,4,5]):
        super().__init__()
        self.n_inp,self.n_emb,self.n_hidden,self.n_layers,self.bidirectional,self.bs,self.device,self.pretrain_mtx,self.padding_idx=\
                            n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,pretrain_mtx,padding_idx
        self.n_out,self.n_filters,self.filter_sizes=n_out,n_filters,filter_sizes
        self.dropout_e,self.dropout,self.dropout_o=dropout_e,dropout,dropout_o
        
        self.create_architecture()
        if pretrain_mtx is not None:
            print (f'initializing glove with {pretrain_mtx.shape}')
            self.initialize_glove()
        self.init_hidden()
        self.criterion=nn.BCEWithLogitsLoss()
    
    def set_dropouts(self, dropout, dropout_o, dropout_e):
        self.dropout, self.dropout_o, self.dropout_e = dropout, dropout_o, dropout_e
    
    
    def freeze_embedding(self):
        self.encoder.weight.requires_grad=False
         
    def unfreeze_embedding(self):
        self.encoder.weight.requires_grad=True

    def initialize_glove(self):
        self.encoder.weight.data.copy_(torch.Tensor(self.pretrain_mtx))
    
    def init_hidden(self):
        # Initialize hidden
        self.hidden=(Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)),
                     Variable(torch.zeros(self.n_layers,self.bs,self.n_hidden,requires_grad=False).to(self.device)))
    
      
    def create_architecture_cnn(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
          # embs are going to be of shape n_batch * n_seq * n_emb
            
            #embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.
        self.conv_0=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[0])
        self.conv_1=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[1])
        self.conv_2=torch.nn.Conv1d(self.n_emb,self.n_filters,kernel_size=self.filter_sizes[2])

        # 
        
        
        
        self.fc=nn.Linear(len(self.filter_sizes)*self.n_filters,self.n_out)
        self.dropout_op=nn.Dropout(self.dropout_o)
                                    
    def create_architecture_lstm(self):
        # Dropout layer
        self.dropout_enc=nn.Dropout(self.dropout_e)
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
             # embs are going to be of shape n_batch * n_seq * n_emb
            # h and c are of shape n_layers * n_batch * n_hidden
        # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
        
                # output is of shape bs * n_seq * n_hidden
            # output [:,-1,:] is the same as hn[-1] but contains the padding idx also. Those are not there for hn[-1]
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        self.avg_pool1d=torch.nn.AdaptiveAvgPool1d(1)
        self.max_pool1d=torch.nn.AdaptiveMaxPool1d(1)
        
        self.fc1=nn.Linear(self.n_hidden*4,self.n_hidden)
        self.fc2=nn.Linear(self.n_hidden*3,self.n_out)

        #self.log_softmax=nn.LogSoftmax()
        self.sigmoid=nn.Sigmoid()

    def create_architecture(self):
        ###################################
        # Embedding layer - common to both
        ###################################
        self.dropout_enc=nn.Dropout(self.dropout_e)
        self.encoder=nn.Embedding(self.n_inp,self.n_emb,padding_idx=self.padding_idx)
        
        #######################################
        # For RNN #############################
        #######################################
        # Embedding Layer: Embedding layer just maps each word to an index. n_inp to n_emb mapping is all it does
            # input to this is of shape n_batch * n_seq
         # LSTM Layer
        self.lstm=nn.LSTM(self.n_emb,self.n_hidden,self.n_layers,batch_first=True,dropout=self.dropout,\
                          bidirectional=self.bidirectional)
          # embs are going to be of shape n_batch * n_seq * n_emb
        self.dropout_op=nn.Dropout(self.dropout_o)
        
        self.avg_pool1d=torch.nn.AdaptiveAvgPool1d(1)
        self.max_pool1d=torch.nn.AdaptiveMaxPool1d(1)
        self.fc1=nn.Linear(self.n_hidden*4,self.n_hidden)
        self.fc2=nn.Linear(self.n_hidden*3,self.n_out)

        #self.log_softmax=nn.LogSoftmax()
        self.sigmoid=nn.Sigmoid()
    
        #######################################
        # For CNN #############################
        #######################################    
        #embedding dimension is the "depth" of the filter and the number of tokens in the sentence is the width.
        self.conv_0=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[0])
        self.conv_1=torch.nn.Conv1d (self.n_emb,self.n_filters,kernel_size=self.filter_sizes[1])
        self.conv_2=torch.nn.Conv1d(self.n_emb,self.n_filters,kernel_size=self.filter_sizes[2])
        self.fc=nn.Linear(len(self.filter_sizes)*self.n_filters,self.n_out)
        self.dropout_op=nn.Dropout(self.dropout_o)
        # 
        
        ###################################
        ## MERGE THE BOTH OF THEM
        
        self.logisticreg=nn.Linear(2,1)
        
    def forward (self,Xb,Yb,Xb_lengths):
        
        ####RNN PORTION
        embs=self.dropout_enc(self.encoder(Xb))
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)
        lstm_out,lengths=pad_packed_sequence(lstm_out,batch_first=True)
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        avg_pool=self.avg_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        max_pool=self.max_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        max_avg=torch.cat([avg_pool,max_pool],dim=1)
        preds_max_avg=self.fc1(max_avg)
        big_out=torch.cat([hidden,preds_max_avg],dim=1)
        preds_rnn=self.fc2(big_out)
        
        #CNN Portion
        new_embs=embs.permute(0,2,1)
        
        conved_0=torch.relu(self.conv_0(new_embs))
        conved_1=torch.relu(self.conv_1(new_embs))
        conved_2=torch.relu(self.conv_2(new_embs))
        
        
        max_pool1d=torch.nn.MaxPool1d(conved_0.shape[2])
        pooled_0=max_pool1d(conved_0).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_1.shape[2])
        pooled_1=max_pool1d(conved_1).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_2.shape[2])
        pooled_2=max_pool1d(conved_2).squeeze(2)

        cat_cnn = self.dropout_op(torch.cat([pooled_0,pooled_1,pooled_2],dim=1))
        preds_cnn= self.fc (cat_cnn)
       
        
        loss_rnn=self.criterion(preds_cnn,Yb.contiguous().float().view(-1,1)) 
        loss_cnn=self.criterion(preds_rnn,Yb.contiguous().float().view(-1,1))
        #return preds_cnn.view(-1), preds_rnn.view(-1),loss_cnn,loss_rnn 

        preds=self.logisticreg(torch.cat([preds_rnn,preds_cnn],1))
        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))
        return preds.view(-1),loss
        
    def forward_cnn(self,Xb,Yb,Xb_lengths):
        embs=self.dropout_enc(self.encoder(Xb))

        embs=embs.permute(0,2,1)
        
        conved_0=torch.relu(self.conv_0(embs))
        conved_1=torch.relu(self.conv_1(embs))
        conved_2=torch.relu(self.conv_2(embs))
        
        
        max_pool1d=torch.nn.MaxPool1d(conved_0.shape[2])
        pooled_0=max_pool1d(conved_0).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_1.shape[2])
        pooled_1=max_pool1d(conved_1).squeeze(2)
        max_pool1d=torch.nn.MaxPool1d(conved_2.shape[2])
        pooled_2=max_pool1d(conved_2).squeeze(2)

        cat_cnn = self.dropout_op(torch.cat([pooled_0,pooled_1,pooled_2],dim=1))
        preds= self.fc (cat_cnn)
       
              
        
        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))
        
        return preds.view(-1), loss                               
                                    
  
                                    
                                    
    def forward_rnn(self,Xb,Yb,Xb_lengths):
        embs=self.dropout_enc(self.encoder(Xb))
        
        
        
        if Xb.size(0) < self.bs:
            self.hidden=(self.hidden[0][:,:Xb.size(0),:].contiguous(),
            self.hidden[1][:,:Xb.size(0),:].contiguous())

        # before lstm call
            # pack_padded_sequence so that padded items in the sequence won't be shown to the LSTM
        packed_embs = pack_padded_sequence(embs,Xb_lengths,batch_first=True, enforce_sorted=False)
        lstm_out,(hidden,cell)=self.lstm(packed_embs)

        # after lstm call
        lstm_out,lengths=pad_packed_sequence(lstm_out,batch_first=True)
            # lstm_out is of shape bs * n_seq * (n_hidden*n_layers)
        
        # lstm_out has the values that are padded
        hidden = self.dropout_op(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        #import pdb
        #pdb.set_trace()
        avg_pool=self.avg_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        max_pool=self.max_pool1d(lstm_out.permute(0,2,1)).view(Xb.size(0),-1)
        
        # concatenate
        max_avg=torch.cat([avg_pool,max_pool],dim=1)
        preds_max_avg=self.fc1(max_avg)
        
        ## Concatenate hidden, average pool and max pool

        big_out=torch.cat([hidden,preds_max_avg],dim=1)
        preds=self.fc2(big_out)
        
              
        
        loss=self.criterion(preds,Yb.contiguous().float().view(-1,1))
        
        return preds.view(-1), loss  

In [113]:
def accuracy_binomial(preds,actual, device="cpu", cutoff=0.5):
    preds=torch.sigmoid(preds)
    zeros=torch.zeros(len(preds)).to(device)
    ones = torch.ones(len(preds)).to(device)

    preds=torch.where(preds>cutoff,ones,zeros)
    correct=torch.round(preds).long()==actual
    return correct.float().sum()/len(correct)

In [114]:
def accuracy_binomial(preds, y, device="cpu", cutoff=0.5):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    y=y.float()
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [115]:
device="cpu"

## Load pretrained model weights

In [116]:
pretrained_lm_weights=pickle.load(open(f'{PATH}/inter/pretrained_lm_weights','rb'))

In [117]:
if 1==0:
    model=torch.load (f'{PATH}/inter/model_awd_lstm')
    pretrained_lm_weights=model.encoder.weight.data.cpu().numpy()
    import pickle
    pickle.dump(pretrained_lm_weights,open(f'{PATH}/inter/pretrained_lm_weights','wb'))

### Check if model forward works

In [118]:
n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout_o,dropout,n_out

(59972, 400, 400, 2, True, 52, 'cpu', 0.2, 0.5, 0.5, 1)

In [119]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e=dropout_e,dropout=dropout,\
                 dropout_o=dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)


initializing glove with (59972, 400)


In [120]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [121]:
print(f'The model has {count_parameters(model_sentiment):,} trainable parameters')

The model has 31,523,805 trainable parameters


In [122]:
model_sentiment

sentiment_classifier(
  (dropout_enc): Dropout(p=0.2)
  (encoder): Embedding(59972, 400, padding_idx=1)
  (lstm): LSTM(400, 400, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout_op): Dropout(p=0.5)
  (avg_pool1d): AdaptiveAvgPool1d(output_size=1)
  (max_pool1d): AdaptiveMaxPool1d(output_size=1)
  (fc1): Linear(in_features=1600, out_features=400, bias=True)
  (fc2): Linear(in_features=1200, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (conv_0): Conv1d(400, 100, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(400, 100, kernel_size=(4,), stride=(1,))
  (conv_2): Conv1d(400, 100, kernel_size=(5,), stride=(1,))
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (logisticreg): Linear(in_features=2, out_features=1, bias=True)
  (criterion): BCEWithLogitsLoss()
)

In [123]:
model_sentiment.forward(xb,yb,xlen)

(tensor([ 0.0908,  0.1028,  0.0493,  0.0428,  0.1904,  0.0555,  0.1753,  0.0766,
          0.1520, -0.0286, -0.0605,  0.0937,  0.1084,  0.1344,  0.0126,  0.1582,
          0.1288,  0.0730,  0.0876,  0.0853, -0.0671,  0.0104,  0.0972, -0.0151,
         -0.0132,  0.1692,  0.2284,  0.1088,  0.1405,  0.0422,  0.0644,  0.1149,
          0.0335,  0.1105,  0.0532,  0.0230,  0.1254,  0.0081,  0.0817,  0.0479,
          0.0725,  0.0325,  0.1215,  0.0334,  0.1429, -0.0116,  0.1447, -0.0355,
          0.1544,  0.1181,  0.1592,  0.1021], grad_fn=<ViewBackward>),
 tensor(0.6885, grad_fn=<BinaryCrossEntropyWithLogitsBackward>))

In [124]:
preds,loss=model_sentiment.forward(xb,yb,xlen)

In [125]:
preds.view(-1)

tensor([-0.0490,  0.0719,  0.2406,  0.0660, -0.0123,  0.1513,  0.0175,  0.0757,
         0.0665, -0.0099,  0.1328,  0.0399,  0.1604,  0.0413,  0.1604,  0.0843,
         0.0206,  0.1009,  0.0219,  0.0296,  0.0690,  0.0601,  0.0097,  0.1480,
         0.0842, -0.0459,  0.1061,  0.0088,  0.1580,  0.1673,  0.1704,  0.0530,
        -0.0396,  0.0483,  0.0504,  0.0242,  0.0242,  0.1843,  0.0028,  0.0637,
         0.1156,  0.0106,  0.1410, -0.0053,  0.1812, -0.0263,  0.1187,  0.0371,
         0.0808, -0.0339,  0.0372,  0.0604], grad_fn=<ViewBackward>)

In [126]:
yb

tensor([0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1,
        0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
        1, 1, 1, 0])

In [127]:
preds.size()

torch.Size([52])

In [128]:
yb.size()

torch.Size([52])

In [129]:
accuracy_binomial(preds.to(device),yb.to(device))

tensor(0.5962)

In [130]:
from sklearn.metrics import roc_auc_score

In [131]:
roc_auc_score(yb,preds)

0.5457271364317842

In [132]:
pretrained_lm_weights

array([[-9.038397e-02, -1.795423e-02, -3.457930e-02,  8.671192e-02, ..., -3.003510e-02, -1.899482e-02,  1.426260e-02,
        -4.948893e-02],
       [-2.453323e-02, -3.018817e-02,  6.663179e-02, -5.793565e-02, ...,  1.840935e-02,  6.371137e-02,  9.835491e-03,
        -2.131385e-03],
       [ 7.696263e-03, -4.266643e-02, -1.533517e-01,  2.239776e-01, ...,  9.869517e-02,  3.041433e-02,  1.824751e-01,
         1.134978e-01],
       [ 3.305928e-02,  2.266591e-01, -4.264669e-02,  1.490862e-01, ...,  3.407921e-02, -6.422209e-03,  3.180612e-01,
         9.373549e-02],
       ...,
       [ 6.027538e-02, -5.980809e-02,  1.861691e-01, -3.105092e-02, ..., -2.764457e-02,  1.962678e-02, -2.172215e-03,
         6.297247e-02],
       [-1.592789e-02, -2.781571e-04,  1.301994e-01,  2.851282e-02, ...,  5.064877e-02,  1.670864e-01,  2.283701e-02,
        -8.746398e-03],
       [ 4.669700e-02,  3.138980e-02,  1.221957e-02, -3.927753e-02, ..., -1.281436e-01,  1.121320e-01, -3.726090e-03,
        -3.777364e

## Learner

In [133]:
class Learner():
    def __init__(self,model,optimizer,metric_fn,device,bptt=12,print_every=5,clip_val=None):
        self.model,self.optimizer,self.metric_fn,self.device,self.print_every,self.bptt,self.losses,self.clip_val=\
            model,optimizer,metric_fn,device,print_every,bptt,[],clip_val
        self.n_epochs=1
  
        
    
    def fit (self,Xb,Yb,Xlen,mode_train=True):
        if mode_train:
            self.model.train()
        else:
            self.model.eval()
            
        preds,loss=self.model(Xb,Yb,Xlen)
        
        with torch.no_grad():
            acc=self.metric_fn(preds,Yb.view(-1),self.device)
            acc=acc.item()
            del preds
        
        if mode_train:
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        myloss=loss.item()
        del loss
        
        if self.clip_val is not None:
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.clip_val)
        
        return myloss, acc
    
    def lr_find (self,start_lr,end_lr,iterator,n_batch):
        losses,lrs=[],[]
        ratio=end_lr/start_lr
        num_steps=n_batch
        lr=start_lr
        for i in range(num_steps):            
            lr=lr*(end_lr/start_lr)**(1/num_steps)
            lrs.append(lr)
        self.lrs=lrs
        self.run_epoch(iterator,mode_train=True,lrs=lrs)
    
    def run_epoch(self,iterator,mode_train,lrs=None):
        epoch_loss,epoch_acc,i,k=0,0,0,0
        self.model.init_hidden()
        for Xb,Yb,Xlen in iterator:
            Xb=Xb.to(self.device)
            Yb=Yb.to(self.device)
            Xlen=Xlen.to(self.device)
            
            if lrs is not None:
                lr=lrs[k]
                for param_group in self.optimizer.param_groups:
                    param_group['lr']=lr 
            

            loss,acc=self.fit(Xb,Yb,Xlen,mode_train)
            
            if lrs is not None:
                self.losses.append(loss)
            
            
            epoch_loss+=loss
            epoch_acc+=acc
            
            k=k+1
            if k%self.print_every == 0:
                if k:
                    print (f'Batch:{k} {epoch_loss/(k)}  {epoch_acc/(k)}')  
                    torch.cuda.empty_cache()
        epoch_loss=epoch_loss/len(iterator)
        epoch_acc=epoch_acc/len(iterator)
            
        return epoch_loss,epoch_acc
    
    def plot_lrs(self, n_roll=1):
        import seaborn as sns
        ax=sns.lineplot(x=self.lrs,y=pd.Series(self.losses).rolling(n_roll).mean())
        ax.set_xscale('log')
        ax.set_ylabel('Loss')
        ax.set_xlabel('Learning Rate')

    
    def run_epochs(self,dltrain,dlvalid,n_epochs=1):
        for epoch in range(n_epochs):
            
            loss,acc=self.run_epoch(dltrain,True)
            print (f'Epoch:{epoch} Loss:{loss}')
            lossv,accv=self.run_epoch(dlvalid,mode_train=False)
            print (f'Epoch:{epoch} Loss:{loss} Accuracy:{acc} Loss:{lossv} Accuracy:{accv}')
        

In [134]:
device="cuda:0"

In [135]:
n_layers

2

In [136]:
model_sentiment=sentiment_classifier (n_inp,n_emb,n_hidden,n_layers,bidirectional,bs,device,dropout_e,dropout,\
                 dropout_o,pretrain_mtx=pretrained_lm_weights,n_out=1,padding_idx=1)
model_sentiment=model_sentiment.to(device)

initializing glove with (59972, 400)


In [137]:
optimizer=torch.optim.Adam(model_sentiment.parameters(),lr=5e-3,betas=(0.9,0.999), weight_decay=wd)
metric_fn=accuracy_binomial

In [138]:
len(dltrain),len(dlvalid)

(481, 481)

In [139]:
df_valid['label'].value_counts()

1    12500
0    12500
Name: label, dtype: int64

In [140]:
learner=Learner(model_sentiment,optimizer,accuracy_binomial,device,bptt,50,0.25)

In [141]:
model_sentiment.freeze_embedding()

In [142]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(False, False)

In [143]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o, learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

(0.2, 0.5, 0.5, 0.2, 0.5, 0.5)

In [ ]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.6920940601825714  0.5596154028177261
Batch:100 0.6403432920575142  0.6261538678407669
Batch:150 0.5677205526828766  0.6838461780548095
Batch:200 0.5236557738482952  0.7186538711190223


In [88]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.2969422796368599  0.883076959848404
Batch:100 0.30535467609763145  0.873846189379692
Batch:150 0.2947099809845289  0.8794872144858042
Batch:200 0.2945930130034685  0.8787500348687172
Batch:250 0.2967405049800873  0.8763077273368836
Batch:300 0.30113854641715687  0.8746795225143432
Batch:350 0.3023779807771955  0.8745055295739855
Batch:400 0.30335031550377606  0.8748558042943477
Batch:450 0.3062936148047447  0.8737179838286506
Epoch:0 Loss:0.3062660424600272
Batch:50 0.25533897399902344  0.9011538815498352
Batch:100 0.24975233837962152  0.8992308080196381
Batch:150 0.24546665102243423  0.9011538867155711
Batch:200 0.24410639192909003  0.9015385013818741
Batch:250 0.2465755670964718  0.9001538860797882
Batch:300 0.25020610007146993  0.8977564495801925
Batch:350 0.25084444512213977  0.8967033353873661
Batch:400 0.25189696880057455  0.895961577296257
Batch:450 0.2543337704241276  0.8949145681328243
Epoch:0 Loss:0.3062660424600272 Accuracy:0.8738565841484467 Loss:0.25330363311601

In [89]:
learner.run_epochs(dltrain,dlvalid,2)

Batch:50 0.2612586760520935  0.8907692694664001
Batch:100 0.2686653547734022  0.891923114657402
Batch:150 0.269331968575716  0.8923077305157979
Batch:200 0.26797251276671885  0.8938461923599244
Batch:250 0.26805242785811423  0.8940769622325897
Batch:300 0.27078387459119163  0.8919872176647187
Batch:350 0.2732546100871904  0.8908791586330959
Batch:400 0.2743937312811613  0.8897115762531758
Batch:450 0.273371879508098  0.8905983283784654
Epoch:0 Loss:0.2750476974676404
Batch:50 0.25235510617494583  0.9084615755081177
Batch:100 0.24394078984856604  0.9063461941480636
Batch:150 0.23914532800515492  0.9089743995666504
Batch:200 0.2367671152204275  0.9108654245734215
Batch:250 0.23835493183135986  0.9083846547603607
Batch:300 0.2389568299551805  0.9066026033957799
Batch:350 0.24093207263520786  0.9061538852964128
Batch:400 0.24173436006531118  0.9052404238283634
Batch:450 0.24402131956484582  0.9039316626389822
Epoch:0 Loss:0.2750476974676404 Accuracy:0.8897409622485821 Loss:0.24454572651217

In [90]:
learner.run_epochs(dltrain,dlvalid,3)

Batch:50 0.2526996849477291  0.906153883934021
Batch:100 0.24210990980267524  0.9075000375509262
Batch:150 0.24906169007221857  0.9008974726994833
Batch:200 0.2509264434874058  0.8994231143593788
Batch:250 0.2495920861363411  0.900000037431717
Batch:300 0.2506700047602256  0.8996154216925303
Batch:350 0.25195729660136357  0.8992857515811921
Batch:400 0.2543211235664785  0.8982211907207965
Batch:450 0.2534333635866642  0.89799149023162
Epoch:0 Loss:0.25387325458491916
Batch:50 0.24999444887042047  0.9007692730426788
Batch:100 0.2381196627020836  0.9040385007858276
Batch:150 0.23106946304440498  0.9080769618352255
Batch:200 0.2290224463492632  0.9095192700624466
Batch:250 0.2315045661330223  0.9073846547603607
Batch:300 0.23349443073074022  0.9061538861195246
Batch:350 0.2334676503070763  0.9058791608469827
Batch:400 0.23487236255779861  0.904903885871172
Batch:450 0.23705482578939863  0.9045726892683241
Epoch:0 Loss:0.25387325458491916 Accuracy:0.8980369795880546 Loss:0.2375039369905689

In [79]:
model_sentiment.dropout_e,model_sentiment.dropout,model_sentiment.dropout_o, learner.model.dropout_e,learner.model.dropout,learner.model.dropout_o

(0.2, 0.5, 0.5, 0.2, 0.5, 0.5)

In [80]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=5e-4

In [81]:
wd

1e-06

In [82]:
for param_group in learner.optimizer.param_groups:
    param_group['weight_decay']=1e-3

In [83]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.17109055951237678  0.9392308056354522
Batch:100 0.16905868373811245  0.9371154230833053
Batch:150 0.17664438232779503  0.9348718333244324
Batch:200 0.1798207350820303  0.9339423456788063
Batch:250 0.1809715024828911  0.9327692685127258
Batch:300 0.1834358198940754  0.9312179865439733
Batch:350 0.18457236400672367  0.9297802577699934
Batch:400 0.18626736279577016  0.9287500384449959
Batch:450 0.1895132341484229  0.9273504655890995
Epoch:0 Loss:0.19055645330470217
Batch:50 0.21328009873628617  0.9219231164455414
Batch:100 0.20903003100305795  0.9209615767002106
Batch:150 0.2022938776264588  0.9224359353383382
Batch:200 0.2003036970458925  0.9242308074235916
Batch:250 0.20359245420992375  0.9223077306747437
Batch:300 0.20705148369073867  0.9202564483880997
Batch:350 0.2082733883176531  0.9190659723963056
Batch:400 0.20749923232942818  0.9200000385940075
Batch:450 0.21062520899706416  0.9186752523316277
Epoch:0 Loss:0.19055645330470217 Accuracy:0.9270910344351849 Loss:0.21082548

In [88]:
for param_group in learner.optimizer.param_groups:
    param_group['weight_decay']=1e-3

In [85]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=5e-5

In [ ]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.22599529027938842  0.9088461911678314
Batch:100 0.22253445766866206  0.9125000387430191
Batch:150 0.21690935830275218  0.9138461907704671
Batch:200 0.21548882938921451  0.9136538833379746
Batch:250 0.21340395921468736  0.9152308073043823
Batch:300 0.2117832124233246  0.9163461921612421
Batch:350 0.21206261696560041  0.9160989395209721
Batch:400 0.21404753772541882  0.9146154230833053
Batch:450 0.2115788251823849  0.9156837994522519


In [89]:
torch.save(model_sentiment.state_dict(),f'{PATH}/inter/sentiment_model_nounfreeze_state_dict_0.924')
torch.save(optimizer.state_dict(),f'{PATH}/inter/sentiment_learner_nounfreeze_state_dict_0.924')
torch.save (model_sentiment,f'{PATH}/inter/sentiment_model_nounfreeze_0.924')
torch.save (optimizer,f'{PATH}/inter/sentiment_optimizer_nounfreeze_0.924')
torch.save (learner,f'{PATH}/inter/sentiment_learner_nounfreeze_0.924')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [95]:
model_sentiment=torch.load(f'{PATH}/inter/sentiment_model_nounfreeze_0.924')
optimizer=torch.load(f'{PATH}/inter/sentiment_optimizer_nounfreeze_0.924')
learner=torch.load(f'{PATH}/inter/sentiment_learner_nounfreeze_0.924')

In [97]:
for param_group in learner.optimizer.param_groups:
    param_group['lr']=5e-5

In [98]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.19316751204431057  0.925384657382965
Batch:100 0.19021366197615863  0.925384657382965
Batch:150 0.18507408944269022  0.9284615794817607
Batch:200 0.18012874262407422  0.9291346555948258
Batch:250 0.18090652391314507  0.9296154243946075
Batch:300 0.18077860546608765  0.9297436296939849
Batch:350 0.180609092350517  0.9303846549987793
Batch:400 0.1805749541055411  0.9303365784883499
Batch:450 0.18168599167631733  0.9298290995756785
Epoch:0 Loss:0.18122180081950404
Batch:50 0.23026843160390853  0.9238461923599243
Batch:100 0.21336775977164507  0.9261538875102997
Batch:150 0.20659958479305107  0.927692346572876
Batch:200 0.20436845902353526  0.9275000405311584
Batch:250 0.20656096981465816  0.9261538863182068
Batch:300 0.20735313262790442  0.9250641421476999
Batch:350 0.2075942880234548  0.9231319076674325
Batch:400 0.20715798472985625  0.9229327318072319
Batch:450 0.20928612556722428  0.9220940562089285
Epoch:0 Loss:0.18122180081950404 Accuracy:0.9302215330813878 Loss:0.20830832

In [94]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.1795610073208809  0.9334615790843963
Batch:100 0.17820095729082822  0.9340385031700135
Batch:150 0.17120263000329336  0.934487220843633
Batch:200 0.17333494383841752  0.9329808095097541
Batch:250 0.17416198022663593  0.93192311668396
Batch:300 0.17471165298173824  0.9318590136369069
Batch:350 0.17421947273824895  0.9318132259164538
Batch:400 0.1726349769625813  0.9328846541047097
Batch:450 0.17152835315300358  0.933333372010125
Epoch:0 Loss:0.17269179193213924
Batch:50 0.23322612002491952  0.9242308080196381
Batch:100 0.21592979423701764  0.9257692712545394
Batch:150 0.2089104947944482  0.9280769622325897
Batch:200 0.20654483183287084  0.927692346572876
Batch:250 0.2095902508124709  0.9263077313899994
Batch:300 0.20991972533985973  0.9253846542040507
Batch:350 0.21033208885895355  0.923022016797747
Batch:400 0.20985105094965548  0.9228846547007561
Batch:450 0.2116426052028934  0.9220513208707174
Epoch:0 Loss:0.17269179193213924 Accuracy:0.9328082907720316 Loss:0.210453276841

In [87]:
torch.save(model_sentiment.state_dict(),f'{PATH}/inter/sentiment_model_nounfreeze_state_dict_0.917')
torch.save(optimizer.state_dict(),f'{PATH}/inter/sentiment_learner_nounfreeze_state_dict_0.917')
torch.save (model_sentiment,f'{PATH}/inter/sentiment_model_nounfreeze_0.917')
torch.save (optimizer,f'{PATH}/inter/sentiment_optimizer_nounfreeze_0.917')
torch.save (learner,f'{PATH}/inter/sentiment_learner_nounfreeze_0.917')

/home/kirana/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type sentiment_classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [88]:
model_sentiment.unfreeze_embedding()

In [90]:
model_sentiment.encoder.weight.requires_grad, learner.model.encoder.weight.requires_grad

(True, True)

In [91]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.1757665029168129  0.9338461911678314
Batch:100 0.17884126387536525  0.9323077297210693
Batch:150 0.17157089300453662  0.9365384995937347
Batch:200 0.1665740057453513  0.9388461914658547
Batch:250 0.16849402721226214  0.9384615759849548
Batch:300 0.16642002327988545  0.9392949092388153
Batch:350 0.16435486163944005  0.9398901465960912
Batch:400 0.1623965109186247  0.9409615746140481
Batch:450 0.16075962524033255  0.9414102921220991
Epoch:0 Loss:0.16170276429217595
Batch:50 0.22108415484428406  0.9123077321052552
Batch:100 0.21340429924428464  0.9176923459768296
Batch:150 0.20557850728432336  0.9205128566424052
Batch:200 0.1989787089265883  0.9236538824439049
Batch:250 0.2013720786422491  0.921923113822937
Batch:300 0.2017382326349616  0.9211538823445639
Batch:350 0.20419149936309883  0.9204945426327842
Batch:400 0.204031760757789  0.9209134986996651
Batch:450 0.20522147292064297  0.9203419176737467
Epoch:0 Loss:0.16170276429217595 Accuracy:0.9410163481369336 Loss:0.2046149781

In [93]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.11615589462220668  0.966538497209549
Batch:100 0.10260744299739599  0.9701923406124116
Batch:150 0.10320262432098389  0.9685897755622864
Batch:200 0.10043475738726557  0.9695192635059356
Batch:250 0.09697593061625957  0.9700769543647766
Batch:300 0.09517815880477429  0.97019233862559
Batch:350 0.09467971860031997  0.9702747559547424
Batch:400 0.09466664283769205  0.9705769535899162
Batch:450 0.09382010911901792  0.9706837913725112
Epoch:0 Loss:0.09396658818045797
Batch:50 0.24926649011671542  0.9138461947441101
Batch:100 0.2419789445772767  0.9176923471689225
Batch:150 0.23338562476138275  0.9180769606431325
Batch:200 0.2237020975537598  0.9211538836359978
Batch:250 0.22683872967958452  0.9197692685127258
Batch:300 0.22746217250823975  0.9194872166713078
Batch:350 0.23043268159031868  0.9186264107908522
Batch:400 0.23017218199558556  0.9188942676782608
Batch:450 0.2322108591016796  0.9183333708180321
Epoch:0 Loss:0.09396658818045797 Accuracy:0.9705901470600691 Loss:0.2312833

In [89]:
learner.run_epochs(dltrain,dlvalid,1)

Batch:50 0.09524211082607507  0.9680769574642182
Batch:100 0.093828199878335  0.9686538773775101
Batch:150 0.0919878846531113  0.9687179811795552
Batch:200 0.09395078460685909  0.9689423388242722
Batch:250 0.09284830520302058  0.9690769546031952
Batch:300 0.09200737948218982  0.9695513131221135
Batch:350 0.09359771828566278  0.9695055254868099
Batch:400 0.09153297192882746  0.9706731064617634
Batch:450 0.09010110232565138  0.971068405840132
Epoch:0 Loss:0.09070706779714248
Batch:50 0.23427742831408976  0.9230769610404969
Batch:100 0.22739623751491309  0.9246154230833054
Batch:150 0.22029945831745862  0.9264102943738302
Batch:200 0.21663093800656497  0.9272115764021873
Batch:250 0.216403607301414  0.927153883934021
Batch:300 0.2173488519154489  0.9263461919625601
Batch:350 0.21949118124055012  0.9253846536363874
Batch:400 0.2178897394472733  0.92600965321064
Batch:450 0.2196268634787864  0.9252137135134803
Epoch:0 Loss:0.09070706779714248 Accuracy:0.9704582101342089 Loss:0.2193625805214

In [179]:
len(dltrain)

433